# Modelo oficial

- **Objetivo:** Identificar se a pessoa quer ou não trocar de trabalho baseado em suas características

- **Métricas**: 
    - Acurácia
    - Recall
    - Precision
    - F1-score

- **Modelo de ML**: Light Gradient Boosting Machine

## 0. Setup

In [1]:
import pandas as pd
import numpy as np

## 1. Carregando os dados

In [2]:
dados = pd.read_csv(filepath_or_buffer = '../data/raw/aug_train.csv')

dados.head(3)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


## 2. Criar as novas features

### 2.1. Agrupar os valores de company_size em PP, P, M e G

In [32]:
def add_feature_company_size(df):
    
    """
    # PP: Até 50 pessoas
    # P: Vai de 50 até 500
    # M: Vai de 500 até 4999
    # G: Acima de 5000
    """
    
    df1 = df.copy()
    
    df1['company_size_cat'] = np.where(dados['company_size'].isin(['<10', '10/49']), 'PP', 
                                       np.where(dados['company_size'].isin(['50-99', '100-500']), 'P',
                                                np.where(dados['company_size'].isin(['500-999', '1000-4999']), 'M',
                                                         np.where(dados['company_size'].isin(['5000-9999', '10000+']), 'GG', 
                                                                  np.nan))))
    
    return df1

### 2.2. Fazer uma feature que divide a quantidade de horas treinadas por 24 (resultados em quantos dias de treinamento ela participou)

In [35]:
def add_feature_training_hours(df):
    
    df1 = df.copy()
    
    df1['days_training_hours'] = df1['training_hours'] / 24
    
    return df1

### 2.3. Criar uma variável categórica que diz se a pessoa é nova ou não no mercado de trabalho. Ex.: Se a pessoa tem 3 ou menos anos de experiência, ela é nova, senão ela é "velha"

In [45]:
def add_feature_experience(df):
    
    df1 = df.copy()
    
    df1['experience_cat'] = np.where(dados['experience'].isin(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9']), 0, 
                                     np.where(dados['experience'].isin(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']), 1, 
                                              np.nan))
    
    return df1

### 2.4. Agrupar os valores de company_type relacionadas a startup

In [50]:
def add_feature_company_type(df):
    
    df1 = df.copy()
    
    df1['company_type_cat'] = np.where(dados['company_type'].isin(['Funded Startup', 'Early Stage Startup']), 1, 
                                     np.where(dados['company_type'].isin(['Pvt Ltd', 'Other', 'Public Sector', 'NGO']), 0, 
                                              np.nan))
    
    return df1

### 2.5. Criando função para identificar nulos em qualquer variável (se for nulo, 1, 0)

In [63]:
def add_feature_null_column(df, col):
    
    df1 = df.copy()
    
    df1['check_null_' + col] = np.where(df1[col].isna(), 1, 0)
    
    return df1

### 2.6. Criando função para identificar nulos em variáveis qualitativas

In [67]:
def add_feature_null_qualitative(df, col):
    
    df1 = df.copy()
    
    df1[col] = np.where(df1[col].isna(), 'Outras', df1[col])
    
    return df1

### 2.7. Criando função para identificar nulos em variáveis quantitativas

In [70]:
def add_feature_null_quantitative(df, col):
    
    df1 = df.copy()
    
    df1[col] = np.where(df1[col].isna(), 99999, df1[col])
    
    return df1

- enrollee_id: excluir
- city: codificar ela de texto -> número inteiro , testar o encoder - tratar os nulos
- city_development_index: manter e tratar os nulos
- gender: testar todos os encoders, tratar os nulos
- relevent_experience: testar todos os encoders, tratar os nulos
- enrolled_university: testar todos os encoders, tratar os nulos
- education_level: testar todos os encoders, tratar os nulos
- major_discipline: testar todos os encoders, tratar os nulos
- experience: testar todos os encoders, tratar os nulos
- company_size: testar todos os encoders, tratar os nulos
- company_type: testar todos os encoders, tratar os nulos
- last_new_job: testar todos os encoders, tratar os nulos
- training_hours: manter e tratar os nulos

## Próximos passos: 

1. Criar o pipeline inicial com os encoders da library category_encoders
2. Criar o modelo
3. criar a cross-validation
...